In [6]:
from unet import resnet50_unet
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from PIL import Image
import keras
from keras import backend as K
import numpy as np

import tensorflow as tf

import random
import copy
import os

In [7]:
NCLASSES = 3
HEIGHT = 416
WIDTH = 416

In [8]:
def generate_arrays_from_file(lines,batch_size):
    # 获取总长度
    n = len(lines)
    i = 0
    while 1:
        X_train = []
        Y_train = []
        # 获取一个batch_size大小的数据
        for _ in range(batch_size):
            if i==0:
                np.random.shuffle(lines)
            name = lines[i].split(';')[0]
            # 从文件中读取图像
            img = Image.open(r"F:\AAshuju\weed\cropweeds\input\train\images" + '\\' + name)
            img = img.resize((WIDTH,HEIGHT))
            img = np.array(img)
            img = img/255
            X_train.append(img)

            name = (lines[i].split(';')[1]).replace("\n", "")
            # 从文件中读取图像
            img = Image.open(r"F:\AAshuju\weed\cropweeds\input\train\masks_n" + '\\' + name)
            img = img.resize((int(WIDTH/2),int(HEIGHT/2)))
            img = np.array(img)
            seg_labels = np.zeros((int(HEIGHT/2),int(WIDTH/2),NCLASSES))
            for c in range(NCLASSES):
                seg_labels[: , : , c ] = (img[:,:] == c ).astype(int)
            seg_labels = np.reshape(seg_labels, (-1,NCLASSES))
            Y_train.append(seg_labels)

            # 读完一个周期后重新开始
            i = (i+1) % n
        yield (np.array(X_train),np.array(Y_train))

In [9]:

# IOU
def Mean_IOU(y_true, y_pred):
    nb_classes = 3 #K.int_shape(y_pred)[-1]
    iou = []
    true_pixels = K.argmax(y_true, axis=-1)
    pred_pixels = K.argmax(y_pred, axis=-1)
    void_labels = K.equal(K.sum(y_true, axis=-1), 0)
    for i in range(0, nb_classes): # exclude first label (background) and last label (void)
        true_labels = K.equal(true_pixels, i) & ~void_labels
        pred_labels = K.equal(pred_pixels, i) & ~void_labels
        inter = tf.to_int32(true_labels & pred_labels)
        union = tf.to_int32(true_labels | pred_labels)
        legal_batches = K.sum(tf.to_int32(true_labels), axis=1)>0
        ious = K.sum(inter, axis=1)/K.sum(union, axis=1)
        iou.append(K.mean(tf.gather(ious, indices=tf.where(legal_batches)))) # returns average IoU of the same objects
    iou = tf.stack(iou)
    legal_labels = ~tf.debugging.is_nan(iou)
    iou = tf.gather(iou, indices=tf.where(legal_labels))
    return K.mean(iou)


In [ ]:

log_dir = "logs/"
# 获取model
model = resnet50_unet(n_classes=NCLASSES,input_height=HEIGHT, input_width=WIDTH)
# model.summary()


In [ ]:
#打开数据集的txt
with open(r"F:\AAshuju\weed\cropweeds\input\train\train.txt","r") as f:
    lines = f.readlines()

# 打乱行，这个txt主要用于帮助读取数据来训练
# 打乱的数据更有利于训练
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)



In [8]:
# 80%用于训练，20%用于估计。
num_val = int(len(lines)*0.2)
num_train = len(lines) - num_val

# 保存的方式，1世代保存一次
checkpoint_period = ModelCheckpoint(
                                log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                                monitor='val_loss', 
                                save_weights_only=True, 
                                save_best_only=True, 
                                period=1
                             )
# 学习率下降的方式，val_loss三次不下降就下降学习率继续训练
reduce_lr = ReduceLROnPlateau(
                        monitor='val_loss', 
                        factor=0.5, 
                        patience=3, 
                        verbose=1
                     )
# 是否需要早停，当val_loss一直不下降的时候意味着模型基本训练完毕，可以停止
early_stopping = EarlyStopping(
                        monitor='val_loss', 
                        min_delta=0, 
                        patience=10, 
                        verbose=1
                    )



In [9]:
# 交叉熵
model.compile(loss = 'categorical_crossentropy',
        optimizer = 'adam',
        metrics = [Mean_IOU])
    


In [9]:
# Runtime custom callbacks
# %% https://github.com/deepsense-ai/intel-ai-webinar-neural-networks/blob/master/live_loss_plot.py
# Fixed code to enable non-flat loss plots on keras model.fit_generator()
import matplotlib.pyplot as plt
from keras.callbacks import Callback
from IPython.display import clear_output

# from matplotlib.ticker import FormatStrFormatter

def translate_metric(x):
    translations = {'acc': "Accuracy", 'loss': "Log-loss (cost function)"}
    if x in translations:
        return translations[x]
    else:
        return x


class PlotLosses(Callback):
    def __init__(self, figsize=None):
        super(PlotLosses, self).__init__()
        self.figsize = figsize

    def on_train_begin(self, logs={}):

        self.base_metrics = [metric for metric in self.params['metrics'] if not metric.startswith('val_')]
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs.copy())

        clear_output(wait=True)
        plt.figure(figsize=self.figsize)

        for metric_id, metric in enumerate(self.base_metrics):
            plt.subplot(1, len(self.base_metrics), metric_id + 1)

            plt.plot(range(1, len(self.logs) + 1),
                     [log[metric] for log in self.logs],
                     label="training")
            if self.params['do_validation']:
                plt.plot(range(1, len(self.logs) + 1),
                         [log['val_' + metric] for log in self.logs],
                         label="validation")
            plt.title(translate_metric(metric))
            plt.xlabel('epoch')
            plt.legend(loc='center left')

        plt.tight_layout()
        plt.show();    

plot_losses = PlotLosses(figsize=(16, 6))

In [10]:
batch_size = 1
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    
# 开始训练
model.fit_generator(generate_arrays_from_file(lines[:num_train], batch_size),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=generate_arrays_from_file(lines[num_train:], batch_size),
        validation_steps=max(1, num_val//batch_size),
        epochs=45,
        initial_epoch=0,
        callbacks=[checkpoint_period, reduce_lr])# plot_losses


Train on 1263 samples, val on 315 samples, with batch size 1.
Epoch 1/45
1263/1263 [==============================] - 789s 624ms/step - loss: 0.0850 - Mean_IOU: 0.6412 - val_loss: 0.0594 - val_Mean_IOU: 0.6081
Epoch 2/45
1263/1263 [==============================] - 746s 590ms/step - loss: 0.0334 - Mean_IOU: 0.7161 - val_loss: 0.0739 - val_Mean_IOU: 0.5891
Epoch 3/45
1263/1263 [==============================] - 747s 591ms/step - loss: 0.0298 - Mean_IOU: 0.7386 - val_loss: 0.0544 - val_Mean_IOU: 0.5894
Epoch 4/45
1263/1263 [==============================] - 747s 592ms/step - loss: 0.0279 - Mean_IOU: 0.7502 - val_loss: 0.0529 - val_Mean_IOU: 0.6292
Epoch 5/45
1263/1263 [==============================] - 747s 592ms/step - loss: 0.0257 - Mean_IOU: 0.7604 - val_loss: 0.0754 - val_Mean_IOU: 0.5791
Epoch 6/45
1263/1263 [==============================] - 747s 592ms/step - loss: 0.0227 - Mean_IOU: 0.7773 - val_loss: 0.0329 - val_Mean_IOU: 0.7613
Epoch 7/45
1263/1263 [============================

#预测

In [14]:
random.seed(0)
class_colors = [[0,0,0],[255,0,0],[0,255,0]]
NCLASSES = 3
HEIGHT = 832
WIDTH = 832

In [17]:
model = resnet50_unet(n_classes=NCLASSES,input_height=HEIGHT, input_width=WIDTH)
model.load_weights("1.h5")

imgs = os.listdir("./img")

In [18]:
for id_ in imgs:

    img = Image.open("./img//"+id_)
    old_img = copy.deepcopy(img)
    orininal_h = np.array(img).shape[0]
    orininal_w = np.array(img).shape[1]

    img = img.resize((WIDTH,HEIGHT))
    img = np.array(img)
    img = img/255
    img = img.reshape(-1,HEIGHT,WIDTH,3)
    pr = model.predict(img)[0]

    pr = pr.reshape((int(HEIGHT/2), int(WIDTH/2),NCLASSES)).argmax(axis=-1)

    seg_img = np.zeros((int(HEIGHT/2), int(WIDTH/2),3))
    colors = class_colors

    for c in range(NCLASSES):
        seg_img[:,:,0] += ( (pr[:,: ] == c )*( colors[c][0] )).astype('uint8')
        seg_img[:,:,1] += ((pr[:,: ] == c )*( colors[c][1] )).astype('uint8')
        seg_img[:,:,2] += ((pr[:,: ] == c )*( colors[c][2] )).astype('uint8')

    seg_img = Image.fromarray(np.uint8(seg_img)).resize((orininal_w,orininal_h))

#     image = Image.blend(old_img,seg_img,0.3)
#     image.save("./img_out/"+id_)
    seg_img.save("./img_out/"+id_)
